In [ ]:
# Import the relevant libraries
 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import copy

In [ ]:
# Deep neural network
# Here I define the model as was written in the MICE article

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      self.conv1 = nn.Conv2d(1, 128, kernel_size=1)
      self.conv2 = nn.Conv2d(128, 64, kernel_size=1)
      self.fc1 = nn.Linear(128, 64)
      self.fc2 = nn.Linear(64, 1)


      '''
      self.conv1 = nn.Conv2d(1, 4, kernel_size=1, stride=1, padding=1)
      self.conv2 = nn.Conv2d(4, 4, kernel_size=1, stride=1, padding=1)
      #self.fc1 = nn.Linear(120, 4000)
      self.fc1 = nn.Linear(48, 4000)
      self.fc2 = nn.Linear(4000, 2)
      '''

    def forward(self, x):
      x = self.conv1(x)
      x = F.relu(x)
      #x = self.conv2(x)
      #x = F.relu(x)
      x = torch.flatten(x, 1)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc2(x)
      output = F.log_softmax(x, dim=1)
      return x

In [ ]:
# The loss function - Mutual information
# Here we define the loss function of the model as the Mutual information, as we have seen in the MICE article
# M_x: is the mutual information as was calculated by the model
# M_y: is the mutual information as was calculated theoretically

def MyLossFunction(output_train_joint, y_train_joint, output_train_product, y_train_product):
  exponent_product = np.exp(output_train_product.detach().numpy())
  average_product = np.sum(exponent_product) / len(exponent_product)
  average_joint = np.sum(output_train_joint.detach().numpy())/len(output_train_joint.detach().numpy())
  M_x = average_joint - np.log(average_product)


  #exponent_product = np.exp(y_train_product.detach().numpy())
  exponent_product = np.exp(output_train_product.detach().numpy())
  average_product = np.sum(exponent_product) / len(exponent_product)

  #average_joint = np.sum(y_train_joint.detach().numpy())/len(y_train_joint.detach().numpy())
  average_joint = np.sum(output_train_joint.detach().numpy())/len(output_train_joint.detach().numpy())
  
  
  M_y = average_joint - np.log(average_product)

  mse = np.mean((M_y - M_x) ** 2)

  return mse, -M_y, -M_x

In [ ]:
# The train function
# Here we train the model with the Adam optimizer while minimizing the loss function - mutual information

def train(model, epoch, joint_tensor, product_tensor, MyLossFunction):

    PATH = './ising_net.pth'

    model.load_state_dict(torch.load(PATH))

    y_train_joint = None
    y_train_product = None
    model.train()
    tr_loss = 0
    if torch.cuda.is_available():
        joint_tensor = joint_tensor.cuda()
        product_tensor = product_tensor.cuda()


    optimizer.zero_grad()
    output_train_joint = model(joint_tensor)
    output_train_product = model(product_tensor)

    loss_train, M_y, M_x = MyLossFunction(output_train_joint, y_train_joint, output_train_product, y_train_product)

    train_losses.append(M_x)

    loss_train = torch.from_numpy(np.array(M_x))

    loss_train.requires_grad=True

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    
    tr_loss = loss_train.item()
    
    torch.save(model.state_dict(), PATH)

  
    '''
    if epoch%2 == 0:

        print('Epoch : ',epoch+1, '\t', 'loss :', tr_loss)
        print(f'Mutual real: {M_y}, Mutual mine: {tr_loss}')
    '''

    
    return model, train_losses, M_y, M_x

In [ ]:
# initialize the ising lattice with randomly zeros and ones

random_lattice = torch.randn((2,4))
ising_lattice = torch.ones((2,4))
ising_lattice[random_lattice < 0.5] = -1
sub_sub_ising_product = []
sub_sub_ising_joint = []

data_len = 5000

In [ ]:
# Finite temperature Metropolis monte carlo
# Here we split the ising lattice into subsystems as was written in the MICE article

J = 1
kT = 3
seed = 1234
RS = np.random.RandomState(seed)

sub_sub_sub_ising_joint = []
sub_sub_sub_ising_product = []

for data in range(data_len):

  sub_sub_ising_joint = []
  sub_sub_ising_product = []

  sub_ising_joint = []
  sub_ising_joint.append((list(torch.split(ising_lattice, 2)))[0][0])
  sub_ising_joint.append((list(torch.split(ising_lattice, 2)))[0][1])
  places = np.linspace(0, 8, num=8, endpoint=False)
  RS.shuffle(places)
  ising_flatten = ising_lattice.flatten()
  sub_ising_product = []
  sub_ising_product.append(torch.tensor([ising_flatten[int(places[0])],
                                         ising_flatten[int(places[1])],
                                         ising_flatten[int(places[2])],
                                         ising_flatten[int(places[3])]]))
  places = np.linspace(0, 8, num=8, endpoint=False)
  RS.shuffle(places)
  sub_ising_product.append(torch.tensor([ising_flatten[int(places[0])],
                                         ising_flatten[int(places[1])],
                                         ising_flatten[int(places[2])],
                                         ising_flatten[int(places[3])]]))
  
  sub_sub_ising_joint += [list(torch.split(sub_ising_joint[i],2))[j]
                          for i in range(2)
                          for j in range(2)]
  for i in range(2):
    for j in range(2):
      sub_sub_ising_joint.append(list(torch.split(sub_ising_joint[i],2))[j])
  
  for i in range(4):
    for j in range(2):
      sub_sub_sub_ising_joint.append(sub_sub_ising_joint[i][j])

  places = np.linspace(0,4,num=4,endpoint=False)
  RS.shuffle(places)

  sub_sub_ising_product.append(torch.tensor([sub_ising_product[0][RS.randint(0, 4)],
                                             sub_ising_product[0][RS.randint(0, 4)]]))
  sub_sub_ising_product.append(torch.tensor([sub_ising_product[0][RS.randint(0, 4)],
                                             sub_ising_product[0][RS.randint(0, 4)]]))
  sub_sub_ising_product.append(torch.tensor([sub_ising_product[1][RS.randint(0, 4)],
                                             sub_ising_product[1][RS.randint(0, 4)]]))
  sub_sub_ising_product.append(torch.tensor([sub_ising_product[1][RS.randint(0, 4)],
                                             sub_ising_product[1][RS.randint(0, 4)]]))

  sub_sub_sub_ising_product.append(sub_sub_ising_product[0][0])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[0][1])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[1][0])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[1][1])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[2][0])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[2][1])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[3][0])
  sub_sub_sub_ising_product.append(sub_sub_ising_product[3][1])
  
# Here we try to switch one spin value, while using T = const according to the Metropolis Monte Carlo algorithm

  places1 = np.linspace(0, 4, num=4, endpoint=False)
  RS.shuffle(places1)
  places2 = np.linspace(0, 2, num=2, endpoint=False)
  RS.shuffle(places2)
  r = int(places2[0])
  c = int(places1[0])

  rowy1 = r + 1
  rowy2 = r - 1
  coly1 = c + 1
  coly2 = c - 1

  if rowy1 >= ising_lattice.shape[0]:
    rowy1 = 0
  
  if coly1 >= ising_lattice.shape[1]:
    coly1 = 0

  # print(rowy1, rowy2, coly1, coly2)

  Ergy_old = J * ising_lattice[r][c] * (ising_lattice[r][coly1] + ising_lattice[r][coly2] + ising_lattice[rowy1][c] + ising_lattice[rowy2][c])
  Ergy_new = (-1) * J * ising_lattice[r][c] * (ising_lattice[r][coly1] + ising_lattice[r][coly2] + ising_lattice[rowy1][c] + ising_lattice[rowy2][c])
  

  if Ergy_new < Ergy_old:
    ising_lattice[r][c] = ising_lattice[r][c] * (-1)

  elif Ergy_new > Ergy_old:
    e = np.exp(-Ergy_new/kT)
    if np.random.rand(1)[0] < e:
      ising_lattice[r][c] = ising_lattice[r][c] * (-1)

In [ ]:
# For not changing the entire code:

joint_list = copy.deepcopy(sub_sub_sub_ising_joint)

product_list = copy.deepcopy(sub_sub_sub_ising_product)

In [ ]:
# Neural Net
# Here we send our subsystems of joint_tensor and product_tensor into our neural network as was written in the MICE algorithm

model = Net()

PATH = './ising_net.pth'

torch.save(model.state_dict(), PATH)

optimizer = Adam(model.parameters(), lr=0.01, weight_decay=0.1)

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    MyLossFunction = MyLossFunction.cuda()
      
n_epochs = 25
# empty list to store training losses
train_losses = []


cntr0 = 0
cntr1 = 128

#n_epochs = 1
# training the model
for epoch in range(n_epochs):

  cntr0 = 0
  cntr1 = 128
  flag = 0

  for batch in range(int((len(sub_sub_sub_ising_product) - (len(sub_sub_sub_ising_product) % 128))/128)):
    model, train_losses, M_y, M_x = train(model, epoch, torch.tensor(joint_list[cntr0:cntr1]).view(-1,1,1,1), torch.tensor(product_list[cntr0:cntr1]).view(-1,1,1,1), MyLossFunction)
    cntr0 += 128
    cntr1 += 128
    flag += 1

    if flag % 10 == 0:
      pass
      #print(f'Number : {flag}, Mutual information: {abs(M_x)}')

  if epoch%2 == 0:

    #pass

    #print('Epoch : ',epoch+1)
    
    print(f'Epoch : {epoch+1}    Mutual information: {abs(M_y)}')
    
    #print('Epoch : ',epoch+1, '\t', 'loss :', tr_loss)
    #print(f'Mutual real: {M_y}, Mutual mine: {tr_loss}')
    #print(f'Mutual real: {M_y}')

Epoch : 1    Mutual information: 0.002353936149926697
Epoch : 3    Mutual information: 0.002353936149926697
Epoch : 5    Mutual information: 0.002353936149926697
Epoch : 7    Mutual information: 0.002353936149926697
Epoch : 9    Mutual information: 0.002353936149926697
Epoch : 11    Mutual information: 0.002353936149926697
Epoch : 13    Mutual information: 0.002353936149926697
Epoch : 15    Mutual information: 0.002353936149926697
Epoch : 17    Mutual information: 0.002353936149926697
Epoch : 19    Mutual information: 0.002353936149926697
Epoch : 21    Mutual information: 0.002353936149926697
Epoch : 23    Mutual information: 0.002353936149926697
Epoch : 25    Mutual information: 0.002353936149926697


In [ ]:
'''

###### Old one #######




# Neural Net
# Here we send our subsystems of joint_tensor and product_tensor into our neural network as was written in the MICE algorithm


joint_tensor = torch.cat(sub_sub_ising_joint,axis=-1).view(-1,2)
product_tensor = torch.cat(sub_sub_ising_product,axis=-1).view(-1,2)
joint_tensor[joint_tensor == 0] = -1
product_tensor[product_tensor == 0] = -1

y_train_joint = joint_tensor[:,0] == joint_tensor[:,1]
y_train_product = product_tensor[:,0] == product_tensor[:,1]

joint_tensor = torch.unsqueeze(joint_tensor, 1)
joint_tensor = torch.unsqueeze(joint_tensor, 2)
product_tensor = torch.unsqueeze(product_tensor, 1)
product_tensor = torch.unsqueeze(product_tensor, 2)

model = Net()

optimizer = Adam(model.parameters(), lr=0.01, weight_decay=0.1)

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    MyLossFunction = MyLossFunction.cuda()
      
n_epochs = 25
# empty list to store training losses
train_losses = []

# training the model
for epoch in range(n_epochs):
    model, train_losses, M_y, M_x = train(model, epoch, joint_tensor, product_tensor, MyLossFunction)

'''

'\n\n###### Old one #######\n\n\n\n\n# Neural Net\n# Here we send our subsystems of joint_tensor and product_tensor into our neural network as was written in the MICE algorithm\n\n\njoint_tensor = torch.cat(sub_sub_ising_joint,axis=-1).view(-1,2)\nproduct_tensor = torch.cat(sub_sub_ising_product,axis=-1).view(-1,2)\njoint_tensor[joint_tensor == 0] = -1\nproduct_tensor[product_tensor == 0] = -1\n\ny_train_joint = joint_tensor[:,0] == joint_tensor[:,1]\ny_train_product = product_tensor[:,0] == product_tensor[:,1]\n\njoint_tensor = torch.unsqueeze(joint_tensor, 1)\njoint_tensor = torch.unsqueeze(joint_tensor, 2)\nproduct_tensor = torch.unsqueeze(product_tensor, 1)\nproduct_tensor = torch.unsqueeze(product_tensor, 2)\n\nmodel = Net()\n\noptimizer = Adam(model.parameters(), lr=0.01, weight_decay=0.1)\n\n# checking if GPU is available\nif torch.cuda.is_available():\n    model = model.cuda()\n    MyLossFunction = MyLossFunction.cuda()\n      \nn_epochs = 25\n# empty list to store training los